<a href="https://colab.research.google.com/github/cagBRT/Recommendation-Systems/blob/main/neural_collaborative_filter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recommender systems - Neural Collaborative Filtering
> Demo

- toc: true
- badges: true
- comments: true
- hide: true
- categories: [demo, neural networks, deep learning, recommender systems, paper]
- image: https://raw.githubusercontent.com/murilo-cunha/inteligencia-superficial/master/images/2020-09-11-neural_collaborative_filter/cover.png

Download dependencies and run `tensorboard` in the background:

```python
!pip install tensorflow lightfm pandas
```

```python
%load_ext tensorboard
!tensorboard --logdir 2020-09-11-neural_collaborative_filter/logs &
```

In [2]:
!pip install lightfm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp310-cp310-linux_x86_64.whl size=808333 sha256=1341db1e623bf98a715d49c2bb72c4c11e6b33b8018ba88972a75cd2ece158eb
  Stored in directory: /root/.cache/pip/wheels/4f/9b/7e/0b256f2168511d8fa4dae4fae0200fdbd729eb424a912ad636
Successfully built lightfm


In [3]:
# hide
import datetime
import os

import lightfm
import numpy as np
import pandas as pd
import tensorflow as tf
from lightfm import LightFM
from lightfm.datasets import fetch_movielens
from scipy import sparse

In [4]:
# hide
print(f"Tensorflow version: {tf.__version__}")
print(f"LightFM version: {lightfm.__version__}")
print(f"Pandas version: {pd.__version__}")
print(f"Numpy version: {np.__version__}")

Tensorflow version: 2.14.0
LightFM version: 1.17
Pandas version: 1.5.3
Numpy version: 1.23.5


In [5]:
# hide
TOP_K = 5
N_EPOCHS = 10

# Data

![](https://raw.githubusercontent.com/murilo-cunha/inteligencia-superficial/master/images/2020-09-11-neural_collaborative_filter/matrix_factorization_with_alpha.png "Credit: https://developers.google.com/machine-learning/recommendation/collaborative/basics")

In [17]:
# hide_input
data = fetch_movielens(min_rating=3.0)


print("Interaction matrix:")
print(data["train"].toarray()[:20, :20])

Interaction matrix:
[[5 3 4 3 3 5 4 0 5 3 0 5 5 5 5 5 3 4 5 0]
 [4 0 0 0 0 0 0 0 0 0 0 0 0 4 0 0 0 0 3 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 4 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [4 0 0 0 0 0 0 4 4 0 0 4 0 0 3 0 0 0 4 0]
 [0 0 0 5 0 0 5 5 5 4 3 5 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 3 0 0 0 3 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 4 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [4 0 0 4 0 0 0 0 4 0 4 5 3 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 4 5 0 0 0 0 0 5 0 0 0 0 0]
 [0 0 0 5 0 0 0 0 0 0 0 0 0 0 5 0 0 0 0 0]
 [3 3 0 5 0 0 0 4 3 0 0 5 5 4 0 0 0 0 0 0]
 [0 0 0 0 0 0 5 0 4 0 0 5 4 3 4 0 0 3 5 0]
 [0 0 0 0 0 0 0 0 4 0 0 0 0 4 4 0 0 0 0 3]
 [5 0 0 5 0 0 5 0 5 0 5 5 0 0 5 0 0 0 0 0]
 [0 0 0 0 0 0 4 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [5 0 0 3 0 5 0 5 5 0 0 5 5 5 4 0 0 0 3 0]
 [0 0 0 0 0 0 0 5 0 0 0 0 0 0 0 0 0 0 0 0]
 [3 0 0 0 0 0 0 0 0 0 0 0 0 0 4 0 0 0 0 0]]


In [22]:
# collapse
for dataset in ["test", "train"]:
    data[dataset] = (data[dataset].toarray() > 0).astype("int8")

# Make the ratings binary
print("Interaction matrix:")
print(data["train"][:10, :10])

print("\nRatings:")
unique_ratings = np.unique(data["train"])
print(unique_ratings)

Interaction matrix:
[[1 1 1 1 1 1 1 0 1 1]
 [1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 1 1 0]
 [0 0 0 1 0 0 1 1 1 1]
 [0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 1 0 0 0]
 [1 0 0 1 0 0 0 0 1 0]]

Ratings:
[0 1]


In [23]:
from typing import List


def wide_to_long(wide: np.array, possible_ratings: List[int]) -> np.array:
    """Go from wide table to long.
    :param wide: wide array with user-item interactions
    :param possible_ratings: list of possible ratings that we may have."""

    def _get_ratings(arr: np.array, rating: int) -> np.array:
        """Generate long array for the rating provided
        :param arr: wide array with user-item interactions
        :param rating: the rating that we are interested"""
        idx = np.where(arr == rating)
        return np.vstack(
            (idx[0], idx[1], np.ones(idx[0].size, dtype="int8") * rating)
        ).T

    long_arrays = []
    for r in possible_ratings:
        long_arrays.append(_get_ratings(wide, r))

    return np.vstack(long_arrays)

In [24]:
long_train = wide_to_long(data["train"], unique_ratings)
df_train = pd.DataFrame(long_train, columns=["user_id", "item_id", "interaction"])

In [25]:
# hide_input
print("All interactions:")
df_train.head()

All interactions:


,user_id,item_id,interaction
0,0,7,0
1,0,10,0
2,0,19,0
3,0,20,0
4,0,26,0


In [26]:
# hide_input
print("Only positive interactions:")
df_train[df_train["interaction"] > 0].head()

Only positive interactions:


,user_id,item_id,interaction
1511499,0,0,1
1511500,0,1,1
1511501,0,2,1
1511502,0,3,1
1511503,0,4,1


# The model (Neural Collaborative Filtering)

<center><img src="https://raw.githubusercontent.com/murilo-cunha/inteligencia-superficial/master/images/2020-09-11-neural_collaborative_filter/ncf_all_with_alpha.png" width="70%" url="https://developers.google.com/machine-learning/recommendation/collaborative/basics" description="Fonte: https://developers.google.com/machine-learning/recommendation/collaborative/basics" /> </center>

In [29]:
import keras as keras
from keras.layers import (
    Concatenate,
    Dense,
    Embedding,
    Flatten,
    Input,
    Multiply,
)
from keras.models import Model
from keras.regularizers import l2


def create_ncf(
    number_of_users: int,
    number_of_items: int,
    latent_dim_mf: int = 4,
    latent_dim_mlp: int = 32,
    reg_mf: int = 0,
    reg_mlp: int = 0.01,
    dense_layers: List[int] = [8, 4],
    reg_layers: List[int] = [0.01, 0.01],
    activation_dense: str = "relu",
) -> keras.Model:

    # input layer
    user = Input(shape=(), dtype="int32", name="user_id")
    item = Input(shape=(), dtype="int32", name="item_id")

    # embedding layers
    mf_user_embedding = Embedding(
        input_dim=number_of_users,
        output_dim=latent_dim_mf,
        name="mf_user_embedding",
        embeddings_initializer="RandomNormal",
        embeddings_regularizer=l2(reg_mf),
        input_length=1,
    )
    mf_item_embedding = Embedding(
        input_dim=number_of_items,
        output_dim=latent_dim_mf,
        name="mf_item_embedding",
        embeddings_initializer="RandomNormal",
        embeddings_regularizer=l2(reg_mf),
        input_length=1,
    )

    mlp_user_embedding = Embedding(
        input_dim=number_of_users,
        output_dim=latent_dim_mlp,
        name="mlp_user_embedding",
        embeddings_initializer="RandomNormal",
        embeddings_regularizer=l2(reg_mlp),
        input_length=1,
    )
    mlp_item_embedding = Embedding(
        input_dim=number_of_items,
        output_dim=latent_dim_mlp,
        name="mlp_item_embedding",
        embeddings_initializer="RandomNormal",
        embeddings_regularizer=l2(reg_mlp),
        input_length=1,
    )

    # MF vector
    mf_user_latent = Flatten()(mf_user_embedding(user))
    mf_item_latent = Flatten()(mf_item_embedding(item))
    mf_cat_latent = Multiply()([mf_user_latent, mf_item_latent])

    # MLP vector
    mlp_user_latent = Flatten()(mlp_user_embedding(user))
    mlp_item_latent = Flatten()(mlp_item_embedding(item))
    mlp_cat_latent = Concatenate()([mlp_user_latent, mlp_item_latent])

    mlp_vector = mlp_cat_latent

    # build dense layers for model
    for i in range(len(dense_layers)):
        layer = Dense(
            dense_layers[i],
            activity_regularizer=l2(reg_layers[i]),
            activation=activation_dense,
            name="layer%d" % i,
        )
        mlp_vector = layer(mlp_vector)

    predict_layer = Concatenate()([mf_cat_latent, mlp_vector])

    result = Dense(
        1, activation="sigmoid", kernel_initializer="lecun_uniform", name="interaction"
    )

    output = result(predict_layer)

    model = Model(
        inputs=[user, item],
        outputs=[output],
    )

    return model

In [30]:
# collapse
from keras.optimizers import Adam

n_users, n_items = data["train"].shape
ncf_model = create_ncf(n_users, n_items)

ncf_model.compile(
    optimizer=Adam(),
    loss="binary_crossentropy",
    metrics=[
        tf.keras.metrics.TruePositives(name="tp"),
        tf.keras.metrics.FalsePositives(name="fp"),
        tf.keras.metrics.TrueNegatives(name="tn"),
        tf.keras.metrics.FalseNegatives(name="fn"),
        tf.keras.metrics.BinaryAccuracy(name="accuracy"),
        tf.keras.metrics.Precision(name="precision"),
        tf.keras.metrics.Recall(name="recall"),
        tf.keras.metrics.AUC(name="auc"),
    ],
)
ncf_model._name = "neural_collaborative_filtering"
ncf_model.summary()

Model: "neural_collaborative_filtering"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 user_id (InputLayer)        [(None,)]                    0         []                            
                                                                                                  
 item_id (InputLayer)        [(None,)]                    0         []                            
                                                                                                  
 mlp_user_embedding (Embedd  (None, 32)                   30176     ['user_id[0][0]']             
 ing)                                                                                             
                                                                                                  
 mlp_item_embedding (Embedd  (None, 32)                   53824     [

In [31]:
def make_tf_dataset(
    df: pd.DataFrame,
    targets: List[str],
    val_split: float = 0.1,
    batch_size: int = 512,
    seed=42,
):
    """Make TensorFlow dataset from Pandas DataFrame.
    :param df: input DataFrame - only contains features and target(s)
    :param targets: list of columns names corresponding to targets
    :param val_split: fraction of the data that should be used for validation
    :param batch_size: batch size for training
    :param seed: random seed for shuffling data - `None` won't shuffle the data"""

    n_val = round(df.shape[0] * val_split)
    if seed:
        # shuffle all the rows
        x = df.sample(frac=1, random_state=seed).to_dict("series")
    else:
        x = df.to_dict("series")
    y = dict()
    for t in targets:
        y[t] = x.pop(t)
    ds = tf.data.Dataset.from_tensor_slices((x, y))

    ds_val = ds.take(n_val).batch(batch_size)
    ds_train = ds.skip(n_val).batch(batch_size)
    return ds_train, ds_val

In [32]:
# create train and validation datasets
ds_train, ds_val = make_tf_dataset(df_train, ["interaction"])

In [ ]:
%%time
# define logs and callbacks
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss", patience=0
)

train_hist = ncf_model.fit(
    ds_train,
    validation_data=ds_val,
    epochs=N_EPOCHS,
    callbacks=[tensorboard_callback, early_stopping_callback],
    verbose=1,
)

Epoch 1/10
2789/2789 [==============================] - 22s 7ms/step - loss: 0.2292 - tp: 2228.0000 - fp: 1556.0000 - tn: 1358858.0000 - fn: 64871.0000 - accuracy: 0.9535 - precision: 0.5888 - recall: 0.0332 - auc: 0.8137 - val_loss: 0.1270 - val_tp: 1301.0000 - val_fp: 657.0000 - val_tn: 150428.0000 - val_fn: 6227.0000 - val_accuracy: 0.9566 - val_precision: 0.6645 - val_recall: 0.1728 - val_auc: 0.9181
Epoch 2/10
2789/2789 [==============================] - 18s 6ms/step - loss: 0.1198 - tp: 13612.0000 - fp: 7921.0000 - tn: 1352493.0000 - fn: 53487.0000 - accuracy: 0.9570 - precision: 0.6321 - recall: 0.2029 - auc: 0.9227 - val_loss: 0.1192 - val_tp: 1603.0000 - val_fp: 884.0000 - val_tn: 150201.0000 - val_fn: 5925.0000 - val_accuracy: 0.9571 - val_precision: 0.6446 - val_recall: 0.2129 - val_auc: 0.9215
Epoch 3/10
2789/2789 [==============================] - 21s 7ms/step - loss: 0.1163 - tp: 14113.0000 - fp: 7941.0000 - tn: 1352473.0000 - fn: 52986.0000 - accuracy: 0.9573 - precision

In [ ]:
long_test = wide_to_long(data["train"], unique_ratings)
df_test = pd.DataFrame(long_test, columns=["user_id", "item_id", "interaction"])
ds_test, _ = make_tf_dataset(df_test, ["interaction"], val_split=0, seed=None)

In [ ]:
%%time
ncf_predictions = ncf_model.predict(ds_test)
df_test["ncf_predictions"] = ncf_predictions

In [ ]:
# hide_input
df_test.head()

In [ ]:
# hide
# sanity checks
# stop execution if low standard deviation (all recommendations are the same)
std = df_test.describe().loc["std", "ncf_predictions"]
if std < 0.01:
    raise ValueError("Model predictions have standard deviation of less than 1e-2.")

In [ ]:
# collapse
data["ncf_predictions"] = df_test.pivot(
    index="user_id", columns="item_id", values="ncf_predictions"
).values
print("Neural collaborative filtering predictions")
print(data["ncf_predictions"][:10, :4])

In [ ]:
precision_ncf = tf.keras.metrics.Precision(top_k=TOP_K)
recall_ncf = tf.keras.metrics.Recall(top_k=TOP_K)

precision_ncf.update_state(data["test"], data["ncf_predictions"])
recall_ncf.update_state(data["test"], data["ncf_predictions"])
print(
    f"At K = {TOP_K}, we have a precision of {precision_ncf.result().numpy():.5f}",
    "and a recall of {recall_ncf.result().numpy():.5f}",
)

In [ ]:
%%time
# LightFM model
def norm(x: float) -> float:
    """Normalize vector"""
    return (x - np.min(x)) / np.ptp(x)


lightfm_model = LightFM(loss="warp")
lightfm_model.fit(sparse.coo_matrix(data["train"]), epochs=N_EPOCHS)

lightfm_predictions = lightfm_model.predict(
    df_test["user_id"].values, df_test["item_id"].values
)
df_test["lightfm_predictions"] = lightfm_predictions
wide_predictions = df_test.pivot(
    index="user_id", columns="item_id", values="lightfm_predictions"
).values
data["lightfm_predictions"] = norm(wide_predictions)

# compute the metrics
precision_lightfm = tf.keras.metrics.Precision(top_k=TOP_K)
recall_lightfm = tf.keras.metrics.Recall(top_k=TOP_K)
precision_lightfm.update_state(data["test"], data["lightfm_predictions"])
recall_lightfm.update_state(data["test"], data["lightfm_predictions"])
print(
    f"At K = {TOP_K}, we have a precision of {precision_lightfm.result().numpy():.5f}",
    "and a recall of {recall_lightfm.result().numpy():.5f}",
)